# Σημειωματάριο πέμπτο: NLP Ετοιμάζοντας τα δεδομένα μας για την βαθύτερη ανάλυση

Σε αυτό το notebook ακολουθούμε όλα τα απαραίτητα βήματα προεπεξεργασίας των δεδομένων κειμένου (δηλαδή των 
tweets μας) πριν τα τροφοδοτήσουμε στην ανάλυσή μας.Δημιουργούμε ακόμη κάποιες νέες στήλες στο DataFrame που μπορεί να μας φανούν χρήσιμα στη συνέχεια (feature extraction). Είναι σημαντικό να σημειωθεί ότι τα βήματα αυτά πρέπει να εκτελεστούν στην σωστή σειρά ώστε να αποφύγουμε την απώλεια πληροφορίας ή την δημιουργία περισσότερου θορύβου στα δεδομένα μας. Για παράδειγμα ένα σημαντικό βήμα στην προεπεξεργασία κειμένου είναι η αφαίρεση των σημείων στίξης. Αυτό όμως πρέπει να γίνει αφού πρώτα διαχειριστούμε τα *#hashtags*, τα *em@ils* και τα *@mentions* γιατί αλλιώς από τη μία θα χάσουμε την πληροφορία που υπάρχει σε τέτοιου είδους μεταδεδομένα κι από την άλλη με την αφαίρεση των σημείων στίξης από αυτά, εκείνο που μένει ενδεχομένως να είναι θόρυβος.
 
Μερικά άρθρα και online υλικό που συμβουλευτήκαμε για την υλοποίηση των βημάτων προεπεξεργασίας είναι τα ακόλουθα: 
- https://towardsdatascience.com/nlp-text-preprocessing-a-practical-guide-and-template-d80874676e79
- https://www.udemy.com/course/nlp-in-python/
- https://www.analyticsvidhya.com/blog/2021/08/a-friendly-guide-to-nlp-text-pre-processing-with-python-example/
- https://machinelearningmastery.com/gentle-introduction-bag-words-model/

In [3]:
%pip install -U pycld2
%pip install emot
%pip install pyspellchecker

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 2.7 MB 372 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import nltk
import re
import string
import unicodedata
import unidecode
import pycld2 as cld2
import emot
from spellchecker import SpellChecker

In [2]:
tweets = pd.read_csv('Data/tweets/tweets_2021_with_users_matched.csv', lineterminator='\n')

Θα δημιουργήσουμε μια νέα στήλη με όνομα *clean_text* όπου θα αποθηκεύονται τα tweets ύστερα από την εφαρμογή των βημάτων preprocessing. Αρχικά διατηρούμε και την στήλη των *tweets* όπου έχουμε τα πρωτότυπα κείμενα έτσι ώστε να μπορούμε να βλέπουμε τις διαφορές που προκύπτουν. Στο τέλος όμως, πριν από την ανάλυση, θα χρειαστεί να αφαιρεθούν.

Στην συνέχεια ακολουθούν ένα ένα τα βήματα του preprocessing.

In [4]:
from textblob import TextBlob

In [5]:
str(TextBlob('I leve u rite me a txt').correct())

'I love u rite me a txt'

In [7]:
%pip install autocorrect


     |████████████████████████████████| 622 kB 637 kB/s eta 0:00:01
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622381 sha256=c93c0443c1555472d313532f47aa4eca9b4109ca64fef0bcedbbbf4a91a51c9a
  Stored in directory: /home/terzisilias/.cache/pip/wheels/ab/0f/23/3c010c3fd877b962146e7765f9e9b08026cac8b035094c5750
Successfully built autocorrect
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import regex
RE_BAD_CHARS = regex.compile(r"\p{Cc}|\p{Cs}")
def remove_bad_chars(text):
    # https://github.com/mikemccand/chromium-compact-language-detector/issues/22
    return RE_BAD_CHARS.sub("", text)

def isEnglish(tweet):
    """
    Έλεγχος για το εάν η γλώσσα του κειμένου του tweet είναι η αγγλική. 
    
        Παράμετροι:
            tweets (str): Το κείμενο του tweet
            
        Επιστρέφει:
            True εάν η μετάφραση που επέστρεψε η βιβλιοθήκη είναι έμπιστη και αν το πιο πολύ κείμενο είναι
            στην αγγλική (οπότε έρχεται πρώτη στο tuple), και εάν το ποσοστό της Αγγλικής γλώσσας 
            στο κείμενο είναι μεγαλύτερο από 90%. (Επιλέχθηκε αυτό το ποσοστό καθώς 
            σε περιπτώσεις μικρού κειμένου ακόμα και αγγλική να είναι 
            η γλώσσα, η βιβλιοθήκη μπορεί να επιστρέψει unknown ή χαμηλό σχετικά ποσοστό.)
            
            False σε κάθε άλλη περίπτωση έτσι ώστε να είμαστε σίγουροι ότι δεν θα κρατήσουμε στο κείμενο προτάσεις
            από άλλες γλώσσες.
    """
    
    tweet = remove_bad_chars(tweet)    
    isReliable, textBytesFound, details = cld2.detect(tweet)
    if isReliable and details[0][0] == 'ENGLISH' and details[0][2] > 90:
        return True
    else:
        return False

In [ ]:
text = '''IT Risk:
Red Hat.python38:3.8 and python38-devel:3.8に複数の脆弱性 -1/2
コードやコマンドを実行される
機密データにアクセスされる
クロスサイトスクリプティング
機密データにアクセスされる
CVE-2021-42771 CVE-2021-33503 CVE-2021-29921 CVE-2021-28957 CVE-2021-23336 CVE-2021-20095'''
cld2.detect(text, returnVectors=True)

In [ ]:
emot_obj = emot.core.emot() 

In [ ]:
def replace_emoticons(text):
    emoticons_found = emot_obj.emoticons(text)['value']
    emoticons_meaning = emot_obj.emoticons(text)['mean']
    for emoticon in emoticons_found:
        text = text.replace(emoticon, emoticons_meaning[emoticons_found.index(emoticon)])
    return text
                      

In [ ]:
def replace_emojis(text):
    emojis_found = emot_obj.emoji(text)['value']
    emojis_meaning = emot_obj.emoji(text)['mean']
    for emoji in emojis_found:
        text = text.replace(emoji, emojis_meaning[emojis_found.index(emoji)].replace(":"," "))
    return text
                      

Δες αυτά παρακάτω. Αυτό σημαίνει ότι πρέπει να διαγράψουμε τους συνδέσμους πρώτα και μετά τα emoticons

In [ ]:
text = '🥰 failure 🚨 :)'

In [ ]:
text_with_emoticon = 'hi there:(sddfgdf)  :D'

In [ ]:
# Lowercase
tweets['clean_text'] = tweets['text'].apply(lambda tweet: tweet.lower())

In [ ]:
# Τακτοποιούμε την νέα στήλη όπου θα αποθηκεύουμε το κείμενο καθώς περνάει από τα διάφορα στάδια προεπεξεργασίας

tweets = tweets.reindex(columns = ['id', 'text', 'clean_text', 'matched_CVE_IDs', 'created_at', 'retweet_count',
       'reply_count', 'like_count', 'quote_count', 'author_id', 'author_name',
       'author_username', 'author_description', 'author_followers_count',
       'author_following_count', 'author_tweet_count', 'author_listed_count'])

In [ ]:
len(tweets[tweets['clean_text'].apply(lambda tweet: not isEnglish(tweet))])

In [ ]:
# Δημιουργία νέας στήλης όπου αποθηκεύουμε το μέγεθος του tweet σε χαρακτήρες

tweets['size_of_tweet_in_characters'] = tweets['text'].apply(lambda tweet: len(tweet))

In [ ]:
# Διαχείριση υπερσυνδέσμων.
# https://stackoverflow.com/a/54086404

# Πρώτα τους παίρνουμε από το text και τους κρατάμε σε μια νέα στήλη
url_pattern = '''(?:(?:https?|ftp):\/\/|\b(?:[a-z\d]+\.))(?:(?:[^\s()<>]+|\((?:[^\s()<>]+|(?:\([^\s()<>]+\)))?\))+(?:\((?:[^\s()<>]+|(?:\(?:[^\s()<>]+\)))?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))?'''
tweets['number_of_links_included'] = tweets['text'].apply(lambda tweet: len(re.findall(url_pattern, tweet)))
tweets['links_included'] = tweets['text'].apply(lambda tweet: re.findall(url_pattern, tweet))


In [ ]:
# Και μετά τους διαγράφουμε από το clean_text
tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: re.sub(url_pattern, '', tweet))

In [ ]:
# Διαχείριση emails

# Πρώτα τα παίρνουμε από το text και τους κρατάμε σε μια νέα στήλη
email_pattern = '([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z+_-]+)'
tweets['number_of_emails_included'] = tweets['text'].apply(lambda tweet: len(re.findall(email_pattern, tweet)))
tweets['emails_included'] = tweets['text'].apply(lambda tweet: re.findall(email_pattern, tweet))


In [ ]:
len(tweets[tweets['number_of_emails_included']>0])

In [ ]:
# Και μετά τα διαγράφουμε από το clean_text
tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: re.sub(email_pattern, '', tweet))

In [ ]:
# Αφαίρεση αχρείαστων κενών (μπορεί να έχουν δημιουργηθεί ξανά)
tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: ' '.join(tweet.split()))

In [ ]:
tweets.head(1)

In [ ]:
# Διαχείριση mentions

# Το εφαρμόζουμε στο clean_text γιατί από εκεί έχουμε αφαιρέσει τα emails οπότε δεν υπάρχει κίμδυνος να πάρουμε 
# και email domains.
# Πρώτα τα παίρνουμε και τα κρατάμε σε μια νέα στήλη
# Χρησιμοποιούμε το replace για να αφαιρέσουμε τπν χαρακτήρα ':' που υπάρχει σε αρκετά mentions. Δεν μπορεί άλλωστε 
# να είναι τμήμα του username καθώς διαβάζουμε ότι "A username can only contain alphanumeric 
# characters (letters A-Z, numbers 0-9) with the exception of underscores"

tweets['number_of_mentions_included'] = tweets['clean_text'].apply(lambda tweet: len([word for word in tweet.split() if word.startswith('@')]))
tweets['mentions_included'] = tweets['clean_text'].apply(lambda tweet: [word.replace(':','') for word in tweet.split() if word.startswith('@')])

In [ ]:
# Και μετά τα διαγράφουμε από το clean_text

tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: ' '.join([word for word in tweet.split() if not word.startswith('@')]))

In [ ]:
tweets[tweets['number_of_mentions_included']>0]

In [ ]:
# Διαγραφή των 'rt'

tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: ' '.join([word for word in tweet.split() if not word == 'rt']))

In [ ]:
# Διαχείριση hashtags
# Πρώτα τα κρατάμε σε μια νέα στήλη 

tweets['number_of_hashtags_included'] = tweets['clean_text'].apply(lambda tweet: len([word for word in tweet.split() if word.startswith('#')]))
tweets['hashtags_included'] = tweets['clean_text'].apply(lambda tweet: [word for word in tweet.split() if word.startswith('#')])

In [ ]:
# Και μετά τα διαγράφουμε από το clean_text

tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: ' '.join([word for word in tweet.split() if not word.startswith('#')]))

In [ ]:
tweets

In [ ]:
tweets[tweets['number_of_hashtags_included']>0]

In [ ]:
tweets.iloc[75555].text

In [ ]:
# https://datagy.io/python-remove-punctuation-from-string/
# Είναι σημαντικό να αφαιρέσουμε τα σημεία στίξης αφού πρώτα διαχειριστούμε υπερσυνδέσμους, hashtags, emojis
# και mentions.
def remove_punctuations(text):
    '''
    Με την συνάρτηση αυτή αφαιρούμε τα σημεία στίξης.
    Η παύλα δεν αφαιρείται για να παραμείνει στο CVE αν και θα ήταν καλό να μπορούμε να αφαιρέσουμε όλες τις παύλες
    που δεν είναι μέρος του CVE.
    '''
    my_punctuations = string.punctuation
    new_text = text.translate(str.maketrans('', ' ', string.punctuation.replace('-','')))
    return new_text

In [ ]:
# Αφαίρεση σημείων στίξης
tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: remove_punctuations(tweet))

In [ ]:
# κειμενο απο αλλες γλωσσες
# τονισμενες λεξεις
# emojis

In [ ]:
def remove_accented_chars(text):
    """Συνάρτηση για αφαίρεση των τόνων, π.χ. από café θα γίνει μετατροπή σε cafe.
       Αυτό είναι απαραίτητο ώστε οι αλγόριθμοι να μην αντιμετωπίσουν ως ξεχωριστές λέξεις δύο ίδιες λόγω διαφορετικού 
       τονισμού.
       SOS: αυτό αφαιρεί και emoji και κείμενο από άλλες γλωσσες και επίσης και άλλους χαρακτήρες όπως # και @ 
       οπότε αφαιρείται."""
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [ ]:
# Παράδειγμα:
x = 'Áccěntěd těxt rks Browser に複数の脆弱性 🚨 NEW: καλημέρα CVE-2021-24014 🚨 Multiple #git @github' 
x = remove_accented_chars(x)
print(x)
' '.join(x.split())

In [ ]:
# υπάρχει και αυτή η επιλογή για το transiteration
unidecode.unidecode("Καλημέρα!")

In [ ]:
# Αφαίρεση τονισμένων, emojis, και κειμένου από άλλες γλώσσες 
tweets["clean_text"] = tweets["clean_text"].apply(lambda tweet: remove_accented_chars(tweet))

In [ ]:
# Αφαίρεση αχρείαστων κενών (μπορεί να έχουν δημιουργηθεί από την αφαίρεση emojis...)
tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: ' '.join(tweet.split()))

In [ ]:
# Tokenization 

tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: nltk.tokenize.word_tokenize(tweet))

In [ ]:
nltk.download('stopwords')

In [ ]:
# Αφαίρεση stopwords

stop_words = set(nltk.corpus.stopwords.words('english'))
tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: [word for word in tweet if word not in stop_words])

In [ ]:
# Stemming

tweets['clean_text'] = tweets['clean_text'].apply(lambda tweet: [nltk.stem.PorterStemmer().stem(word) for word in tweet])

In [ ]:
#pipeline για το model

In [ ]:
# Απομένουν:

# διαχείριση αριθμών (εκτός από αυτών που είναι σε CVE)
# καλύτερα θα ήταν να μεταφράζουμε το κείμενο άλλων γλωσσών που δεν είναι σε λατινικούς χαρακτήρες αντί να το διαγράφουμε; 
# καλύτερα θα ήταν να "μεταφράζουμε" τα emojis;
# χρειάζονται τα Common words removal  και Rare words removal  πρίν το μοντέλο; (από ινδό)
# Spelling Correction ? (από το notebook του ινδού)
# κάνε και ένα wordcloud τώρα που είναι φτιαγμένα τα data

In [ ]:
tweets.to_csv('Data/tweets/tweets_2021_with_users_matched_with_text_preprocessed.csv', index=False)